## 3.1 线性回归
回归(regression)是能为一个或多个自变量与因变量之间关系建模的一类方法。在自然科学和社会科学领域，回归经常用来表示输入和输出之间的关系。  
在机器学习领域中的大多数任务通常都与预测(prediction)有关。当我们想预测一个数值时，就会涉及到回归问题。常见的例子包括：预测价格(房屋、股票等)、预测住院时间(针对住院病人等)、预测需求(零销售等)。但不是所有的预测都是回归问题。在后面我们会介绍分类问题。分类问题的目标是预测数据属于一组类别中的哪一个

### 3.1.1 线性回归的基本元素
线性回归(linear regression)可以追溯19世纪初，它在回归的各种标准工具中最简单且最流行。线性回归基于几个简单的假设：首先，假设自变量$x$和因变量$y$之间的关系是线性，即$y$可以表示为$x$中元素的加权和，这里通常允许包含观测值的一些噪声；其次，我们假设任何噪声都比较正常，如噪声遵循正态分布。

为了解释线性回归，我们举一个实际的例子：我们希望根据房屋的面积(平方英尺)和房龄(年)来估算房屋价格(美元)。为了开发一个能预测房价的模型，我们需要收集一个真实的数据集。这个数据集包括了房屋的销售价格、面积和房龄。在机器学习的术语中，该数据称为训练数据集(training data set)或训练集(training set)。每行数据(比如一次房屋交易相对应的数据)称为样本(sample)，也可以称为数据点(data point)或数据样本(data instance)。我们把试图预测的目标(比如预测房屋价格)称为标签(label)或目标(target)。预测所依据的自变量(面积和房龄)称为特征(feature)或协变量(covariate)。

通常，我们使用$n$来表示数据集中的样本数。对索引为$i$的样本，其输入表示为$x^{(i)}= [x_1^{(i)}, x_2^{(i)}]^T$，其对应的标签是$y^{(i)}$

**线性模型**

线性假设是指目标(房屋价格)可以表示为特征(面积和房龄)的加权和，如下面的式子：
$$
price = w_{area} \cdot area + w_{age} \cdot age + b
$$
其中，$w_{area}$和$w_{age}$称为权重(weight)，权重决定了每个特征对我们预测值的影响。$b$称为偏置(bias)、偏移量(offset)或截距(intercept)。偏置是指当所有特征都取值为0时，预测值应该为多少。即使现实中不会有任何房子的面积是0或房龄是0年，我们仍然需要偏置项。如果没有偏置项，我们模型的表达能力将受到限制。严格来说，上面的公式是输入特征的一个仿射变换(affine transformation)。仿射变换的特点是通过加权和对特征进行线性变换(linear transformation)，并通过偏置项来进行平移(translation)。

给定一个数据集，我们的目标是寻找模型的权重$w$和偏置$b$，使得根据模型做出的预测大体符合数据里的真实价格。输出的预测值由输入特征通过线性模型的仿射变换决定，仿射变换由所选权重和偏置确定。

而在机器学习领域，我们常用的是高维数据，建模时采用线性代数表示法会比较方便。当我们的输入包含$d$个特征时，我们将预测结果$\hat y$(通常使用“尖角”符号表示$y$估计值)表示为：

$$
\hat y = w_1 x_1 + ... + w_d x_d + b
$$

将所有特征放到向量$x \in \mathbb R^d$中，并将所有权重放到向量$w \in \mathbb R^d$中，我们可以用点积形式来简洁地表达模型：

$$
\hat y = w^T x + b
$$

向量$x$对应于单个数据样本的特征。用符号表示的矩阵$X \in \mathbb R^{n \times d}$可以很方便地引用我们整个数据集的$n$个样本。其中，$X$的每一行是一个样本，每一列是一种特征。
对于特征集合$X$，预测值$\hat y \in \mathbb R^n$可以通过矩阵-向量乘法表示为：
$$
\hat y = Xw + b
$$

这个过程中的求和将使用广播机制，给定训练数据特征$X$和对应的已知标签$y$，线性回归的目标是找到一组权重向量$w$和偏置$b$：当给定从$X$的同分布中取样的新样本特征时，这组权重向量和偏置能够使得新样本预测标签的误差尽可能小。

虽然我们相信给定$x$预测$y$的最佳模型会是线性的，但我们很难找到一个有$n$个样本的真实数据集，其中对于所有的$1≤i≤n, y^{(i)}$完全等于$w^T x^{(i)} +b$。无论我们使用什么手段来观察特征$X$和标签$y$，都可能会出现少量的观测误差。因此，即使确信特征与标签的潜在关系是线性的，我们也会加入一个噪声项来观测误差带来的影响。

在开始寻找最好的模型参数(model parameters)$w$和$b$之前，我们还需要两个东西：(1)一种模型质量度量的方式；(2)一种能够更新模型以提高模型预测质量的方法

**损失函数** 

在我们开始考虑如何用模型拟合(fit)数据之前，我们需要确定一个拟合程度的度量。损失函数(loss function)能够量化目标的实际值与预测值之间的差距。通常我们会选择非负数作为损失，且数值越小表示损失越小，完美预测时的损失为0。回归问题中最常用的损失函数是平方误差函数。当样本$i$的预测值为$\hat y^{(i)}$，其相应的真实标签为$y^{(i)}$时，平方误差可以定义为以下公式：

$$
l^{(i)} (w, b) = \frac {1}{2} (\hat y^{(i)}- y^{(i)})^2
$$

常数$\frac{1}{2}$不会带来本质的差别，但这样在形式上稍微简单一些(因为当我们对损失函数求导后常系数为1)。由于训练数据集并不受我们控制，所以经验误差只是关于模型参数的函数。为了进一步说明，来看下面的例子。我们为一维情况下的回归问题绘制图像，如图3.1.1所示。

<div align=center>
<img src='../../pics/3_1_1.jpeg' width='30%'>
</div>

由于平方误差函数中的二次方项，估计值$\hat y^{(i)}$和观测值$y^{(i)}$之间较大的差异将导致更大的损失。为了度量模型在整个数据集上的质量，我们需要计算在训练集$n$个样本的损失均值(也等价于求和)。
$$
L(w, b) = \frac {1}{n} \sum_{i=1}^n l^{(i)}(w, b) = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{2} (w^T x^{(i)} + b -y^{(i)})^2
$$

在训练模型时，我们希望寻找一组参数$(w^*, b^*)$，这组参数能最小化在所有训练样本上的总损失。如下式：
$$
w^*, b^* = \mathop{argmin} \limits_{w,b} L(w,b)
$$

**解析解**

线性回归刚好是一个很简单的优化问题。与我们将在这本书中所讲到的其他大部分模型不同，线性回归的解可以用一个公式简单地表达出来，这类解叫作解析解(analytical solution)。首先，我们将偏置$b$合并到参数$w$中，合并方法是在包含所有参数的矩阵中附加一列。我们的预测问题是最小化$\mid \mid y - Xw \mid \mid ^2$。这在损失平面上只有一个临界点，这个临界点对应于整个区域的损失极小点。将损失关于$w$的导数设为0，得到解析解：

$$
w^* = (X^T X)^{-1} X^T y
$$

像线性回归这样的简单问题存在解析解，但不是所有问题都存在解析解。解析解可以很好的数学分析，但解析解对问题的限制很严格，导致它无法应用于深度学习


**随机梯度下降**  

即使我们无法得到解析解的情况下，我们仍然可以有效地训练模型。在许多任务上，那些难以优化的模型效果要更好。因此，弄清楚如何训练这些难以优化的模型是非常重要的。

本书中我们使用到一种名为梯度下降的(gradient descent)的方法，这种方法几乎可以优化所有的深度学习模型。它通过不断地在损失函数递减的方向上更新参数来降低误差。

梯度下降最简单的用法是计算损失函数(数据集中所有样本的损失均值)关于模型参数的导数(在这里也称为梯度)。但实际中的执行可能会非常慢：因为在每一次更新参数之前，我们必须遍历整个数据集。因此，我们通常会在每次需要计算更新的时候随机抽取一小批样本，这种变体叫做小批量梯度下降(mini batch stochastic gradient descent)。

在每次迭代中，我们首先随机抽样一个小批量$\beta$，它是由固定数量的训练样本组成的。然后，我们计算小批量的平均损失关于模型参数的梯度。最后，我们将梯度乘以一个预先确定的正数$\eta$，并从当前参数的值中减掉。

我们用下面的数学公式来表示这一更新过程($\alpha$表示偏导数)：
$$
(w, b) \leftarrow (w, b) - \frac{\eta}{\mid \beta \mid} \sum_{i \in B} \partial_{w, b} l^{(i)}(w,b)
$$

总结一下，算法的步骤如下：(1)初始化模型参数的值，如随机初始化；(2)从数据集中随机抽取小批量样本且在负梯度的方向上更新参数，并不断迭代这一步骤。对于平方损失和仿射变换，我们可以明确地写成如下形式：
$$
(w, b) \leftarrow (w, b) - \frac{\eta}{\mid \beta \mid} \sum_{i \in B} \partial_{w} l^{(i)}(w,b) = w - \frac{\eta}{\mid \beta \mid} \sum_{i \in B} x^{(i)}(w^T x^{(i)} + b - y^{(i)}) \\
b \leftarrow b - \frac{\eta}{\mid \beta \mid} \sum_{i \in B} \partial_{b} l^{(i)}(w,b) = b - \frac{\eta}{\mid \beta \mid} \sum_{i \in B} x^{(i)}(w^T x^{(i)} + b - y^{(i)})
$$

$w$和$x$都是向量。在这里，更优雅的向量表示法比系数表示法(如 $w_1, w_2, ..., w_d$)更具可读性。$\mid \beta \mid$表示每个小批量中的样本数，这也称为批量大小(batch size)。$\eta$表示学习率(learning rate)。批量大小和学习率的值通常是手动预先指定，而不是通过模型训练得到的。这些可以调整但不在训练过程中更新的参数称为超参数(hyperparameter)。调参(hyperparameter tuning)是选择超参数的过程。超参数是我们根据训练迭代结果来调整的，而训练迭代结果是在独立的验证数据集(validation dataset)上评估得到的。

在训练了预先确定的若干迭代次数后(或者直到满足某些其他停止条件后)，我们记录下模型参数的估计值，表示为$\hat w, \hat b$。但是，即使我们的函数确实是线性的且无噪声，这些估计值也不会使损失函数真正地达到最小值。因为算法会使得损失向最小值缓慢收敛，但却不能在有限的步数内非常精确地达到最小值。

线性回归恰好是一个在整个域只有最小值的学习问题。但是对于深度神经网络这样复杂模型来说，损失平面上通常包含多个最小值。深度学习实践者很少会去话费大力气寻找这样一组参数，使得在训练集上的损失达到组小。事实上，更难做到的是找到一组参数，这组参数能够在我们从未见过的数据上实现较低的损失，这一挑战被称为泛化(generalization)。

**用模型进行预测**  

给定“已学习”的线性回归模型$\hat w^T x + \hat b$，现在我们可以通过房屋面积$x_1$和房龄$x_2$来估计一个(未包含在训练数据中的)新房屋价格。给定特征估计目标的过程通常称为预测(prediction)或推断(inference)。


### 3.1.2 矢量化加速
在训练我们的模型时，我们经常希望能够同时处理整个小批量的样本，为了实现这一点，需要我们对计算进行矢量化，从而利用线性代数库，而不是在Python中编写开销高昂的`for`循环。

In [1]:
%matplotlib inline
import math
import time 
import numpy as np
import torch
from d2l import torch as d2l

为了说明矢量化为什么如此重要，我们考虑对向量相加的两种方法。我们实例化两个全为1的10000维向量。在一种方法中，我们将使用Python的for循环遍历向量；在另一种方法中，我们将依赖对+的调用

In [2]:
n = 10000
a = torch.ones(n)
b = torch.ones(n)

In [3]:
#定义一个计时器
class Timer: #@save
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        self.tik = time.time()

    def stop(self):
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        return sum(self.times) / len(self.times)

    def sum(self):
        return sum(self.times)

    def cumsum(self):
        return np.array(self.times).cumsum().tolist()

In [4]:
#首先，我们使用for循环，每次执行一位的加法
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]

f'{timer.stop():.5f} sec'    

'0.08485 sec'

In [5]:
#或者我们使用重载的+运算符来计算元素的和
timer.start()
d = a + b
f'{timer.stop():.5f} sec'

'0.00037 sec'

结果很明显，第二种方法比第一种方法快得多。矢量化代码通常会带来数量级的加速。

### 3.1.3 正态分布与平方损失